In [4]:
import tensorflow as tf
#import tensorflow.contrib.eager as tfe
import os
import matplotlib.pyplot as plt
import re
import numpy as np
from string import punctuation
from collections import defaultdict
from functools import reduce
from keras.preprocessing.sequence import pad_sequences
from itertools import chain
from InputPreparator import EmbeddingsPreparator
from InputPreparator import StoryParser

/Users/sdoneva/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Input preparation

In [21]:
def get_task_files(task_nr):
    if task_nr==5:
        return 'qa5_three-arg-relations_train.txt', "qa5_three-arg-relations_test.txt"
    if task_nr==6:
        return 'qa6_yes-no-questions_train.txt', 'qa6_yes-no-questions_test.txt'
    if task_nr==10:
        return 'qa10_indefinite-knowledge_train.txt', 'qa10_indefinite-knowledge_test.txt'

In [22]:
train_set_file = get_task_files(10)[0]
test_set_file = get_task_files(10)[1]

train_set_post_file = "tasks_1-20_v1-2/en/"+train_set_file
test_set_post_file = "tasks_1-20_v1-2/en/"+test_set_file

In [23]:
embedder=EmbeddingsPreparator()
story_parser=StoryParser()

In [24]:
vocab_tokens = embedder.get_unique_tokens([train_set_post_file, test_set_post_file])

In [25]:
word_to_index, index_to_embedding = embedder.load_embedding_from_disks("glove.6B.50d.txt",vocab_tokens, with_indexes=True)

In [26]:
train_stories=story_parser.get_stories(train_set_post_file, True)
test_stories=story_parser.get_stories(test_set_post_file, True)

/Users/sdoneva/anaconda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [27]:
contexts_train, questions_train, answers_train = story_parser.vectorize_stories(train_stories, word_to_index)
contexts_test, questions_test, answers_test = story_parser.vectorize_stories(test_stories, word_to_index)

In [28]:
print('contexts.shape = {}'.format(contexts_train.shape))
print('questions.shape = {}'.format(questions_train.shape))
print('answers.shape = {}'.format(answers_train.shape))

contexts.shape = (1000,)
questions.shape = (1000, 6)
answers.shape = (1000, 27)


In [45]:
index_to_embedding.shape[1]

50

In [49]:
contexts_train.shape[1]

IndexError: tuple index out of range

In [29]:
final_train_data = story_parser.get_final_dataset(contexts_train, questions_train, answers_train)
final_test_data = story_parser.get_final_dataset(contexts_test, questions_test, answers_test)

In [30]:
def prep_batch(batch_data, all_data= False, train= False):
    contextsvs, questionsvs, answers=zip(*batch_data)
    
    #Pad to longest sequence in the batch 
    contexts = list(contextsvs)
    max_context_length = max([len(x) for x in contexts])
    questions = list(questionsvs)
    max_query_length = max(len(x) for x in questionsvs)

    final_contexts=pad_sequences(contextsvs, maxlen=max_context_length) 
    final_queries=pad_sequences(questionsvs, maxlen=max_query_length)
    
    return final_contexts, final_queries

# Model

In [50]:
vocab_size=len(index_to_embedding)
final_contexts, final_queries= prep_batch(final_train_data)
q_shape=final_queries.shape[1]
c_shape=final_contexts.shape[1]

In [51]:
q_in = tf.keras.Input(shape=(q_shape,))
c_in = tf.keras.Input(shape=(c_shape,))

#embed
q= tf.keras.layers.Embedding(vocab_size, 50)(q_in)
c= tf.keras.layers.Embedding(vocab_size, 50)(c_in)

#encode
grucell= tf.keras.layers.GRUCell(50)
c = tf.keras.layers.RNN(grucell)(c)
encoded_sentence=tf.keras.layers.Dropout(0.5)(c)

q = tf.keras.layers.RNN(grucell)(q)
encoded_question=tf.keras.layers.Dropout(0.5)(q)

#merge
merged= tf.keras.layers.concatenate([encoded_sentence, encoded_question])

#predict
pred=tf.keras.layers.Dense(answers_train.shape[1], activation=tf.nn.softmax)(merged)

model = tf.keras.Model(inputs=[q_in, c_in], outputs=pred)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 6)            0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 10, 50)       1350        input_8[0][0]                    
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 6, 50)        1350        input_7[0][0]                    
__________________________________________________________________________________________________
rnn_7 (RNN

## Train

In [38]:
model.fit([final_queries, final_contexts], answers_train, epochs=50, batch_size=128)

Epoch 1/50
1000/1000 [==============================] - 2s 2ms/step - loss: 2.5160 - acc: 0.2750
Epoch 2/50
1000/1000 [==============================] - 0s 176us/step - loss: 1.5636 - acc: 0.4460
Epoch 3/50
1000/1000 [==============================] - 0s 189us/step - loss: 1.3509 - acc: 0.4530
Epoch 4/50
1000/1000 [==============================] - 0s 198us/step - loss: 1.1899 - acc: 0.4580
Epoch 5/50
1000/1000 [==============================] - 0s 193us/step - loss: 1.2018 - acc: 0.4540
Epoch 6/50
1000/1000 [==============================] - 0s 254us/step - loss: 1.1929 - acc: 0.4300
Epoch 7/50
1000/1000 [==============================] - 0s 230us/step - loss: 1.1642 - acc: 0.4500
Epoch 8/50
1000/1000 [==============================] - 0s 190us/step - loss: 1.1128 - acc: 0.4420
Epoch 9/50
1000/1000 [==============================] - 0s 190us/step - loss: 1.0848 - acc: 0.4420
Epoch 10/50
1000/1000 [==============================] - 0s 184us/step - loss: 1.0300 - acc: 0.4500
Epoch 11/50

# Test

In [39]:
final_context_test, final_queries_test= prep_batch(final_test_data)

In [40]:
score = model.evaluate([final_queries_test,final_context_test], answers_test, batch_size=128)

1000/1000 [==============================] - 0s 338us/step


In [41]:
score

[0.7028324985504151, 0.49000000286102297]